In [46]:
import markdown
import urllib2
import re
import pandas as pd

In [8]:
currURL = 'https://raw.githubusercontent.com/sindresorhus/awesome/master/readme.md'
# Assign the open file to a variable
webFile = urllib2.urlopen(currURL)

# Read the file contents to a variable
file_contents = webFile.read()


In [49]:
from markdown.treeprocessors import Treeprocessor

class CodeBlockTweaker(Treeprocessor):
    def __init__(self, md):
        self.cur_level = []
        self.last_a = False
        self.output = []
        super(CodeBlockTweaker, self).__init__(md)
    def run(self, root):
        return self.setClass(root)
    def setClass(self, element):
        for child in element:
            if child.tag == "a":
                #print(self.cur_level, child.text, child.get("href"))
                #Out put only github repos
                
                href = child.get("href")                    
                is_repo = re.match('^https://github.com/\w+', href)
                if is_repo is not None:                    
                    self.output.append((self.cur_level + [child.text], href))
                self.last_a = child.text
            # run recursively on children
            if child.tag in ["h2", "h3"]:
                if  self.cur_level:
                    self.cur_level.pop()
                self.cur_level.append(child.text)
                self.last_a = False
            elif child.tag == "ul" and self.last_a:
                self.cur_level.append(self.last_a)
                
            child = self.setClass(child)
            
            if child.tag == "ul" and self.last_a and self.cur_level:                
                self.cur_level.pop()
        return element

from markdown.extensions import Extension

class CodeBlockExtension(Extension):
    def extendMarkdown(self, md, md_globals):
        self.tweaker = CodeBlockTweaker(md)
        md.treeprocessors.add('codeblocktweaker', self.tweaker, '_end')

ext = CodeBlockExtension()
        
html = markdown.markdown(text=unicode(file_contents, 'utf-8'), extensions=[ext])

index_repos = pd.DataFrame(ext.tweaker.output, columns=["categories", "repo_url"])
    

In [12]:
#input_file = codecs.open("some_file.txt", mode="r", encoding="utf-8")
#text = input_file.read()
#html = markdown.markdown(unicode(file_contents, 'utf-8'))

In [51]:
index_repos.describe()

,categories,repo_url
count,327,327
unique,327,327
top,"[Books, R Books]",https://github.com/wbkd/awesome-d3
freq,1,1
